## Проект e-learning. 

Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на вопросы (далее, в ячейках)

Импортируем необходимые библиотеки

In [1]:
import requests
from urllib.parse import urlencode
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
sns.set (
    font_scale =2,
    style      ="whitegrid",
    rc         ={'figure.figsize':(20,7)}
)

Сначала произведем загрузку файлов в наши ДФ с яндекс диска.

*(Сейчас, при подготовке к выгрузке материалов к себе на GitHub я вижу, что 4 ячейки ниже надо было оптимизировать, запихнув их в отдельную функцию и просто передавать ей параметр public_key)*

In [2]:
# Любезно заимствуем код на stackoverflow для загрузки файлов с ЯД, чуть дорабатываем его.
# Загружаем первый файл
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/PBW7aUHGuodFDA'  # Наша ссылка
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
assessments = pd.read_csv(download_url) # Загружаем файл по ссылке в пандас
assessments.head(3)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0


In [3]:
# Второй файл
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/m0Z6QYNT46f9tQ'  
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
courses = pd.read_csv(download_url)
courses.head(3)

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268


In [4]:
# Третий файл
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/lsmdbYB0iM7p3w'  
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
studentAssessment = pd.read_csv(download_url)
studentAssessment.head(3)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0


In [5]:
# Четвертый файл
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/Yse4Y6RJqg_WaA'  
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
studentRegistration = pd.read_csv(download_url)
studentRegistration.head(3)

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0


## Вопрос №1 
Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

Объединим ДФ со студенческими оценками и датафрейм с информацией об оценках в тестах

In [6]:
FullStudentAssessment = studentAssessment.merge(assessments, how='left', on='id_assessment')
FullStudentAssessment

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
0,1752,11391,18,0,78.0,AAA,2013J,TMA,19.0,10.0
1,1752,28400,22,0,70.0,AAA,2013J,TMA,19.0,10.0
2,1752,31604,17,0,72.0,AAA,2013J,TMA,19.0,10.0
3,1752,32885,26,0,69.0,AAA,2013J,TMA,19.0,10.0
4,1752,38053,19,0,79.0,AAA,2013J,TMA,19.0,10.0
...,...,...,...,...,...,...,...,...,...,...
173907,37443,527538,227,0,60.0,GGG,2014J,CMA,229.0,0.0
173908,37443,534672,229,0,100.0,GGG,2014J,CMA,229.0,0.0
173909,37443,546286,215,0,80.0,GGG,2014J,CMA,229.0,0.0
173910,37443,546724,230,0,100.0,GGG,2014J,CMA,229.0,0.0


Узнаем, какие типы оценок бывают

In [7]:
FullStudentAssessment.assessment_type.unique()

array(['TMA', 'CMA', 'Exam'], dtype=object)

Отбираем только сдавших экзамен (score >= 40)

In [8]:
exam_ok = FullStudentAssessment.query('assessment_type == "Exam" and score >= 40')
exam_ok.head()

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
52924,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0
52925,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0
52926,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0
52927,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0
52928,24290,560494,230,0,92.0,CCC,2014B,Exam,NaN,100.0


Сгруппируем по id студента и выявим только одну сдачу экзамена. 

In [9]:
exam_ok.groupby('id_student', as_index = False) \
    .agg({'code_presentation': 'count'}) \
    .query('code_presentation == 1') \
    .shape[0]

3802

***Ответ на первый вопрос - 3 802 человека успешно сдали только один курс***

## Вопрос №2
Выяви самый сложный и самый простой экзамены: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью. (кол-во успешных экзаменов / кол-во всех попыток сдать экзамен)

In [10]:
exams = FullStudentAssessment.query('assessment_type == "Exam"') \
    .groupby('code_module', as_index = False) \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'total_try'}) \
    .merge(exam_ok.groupby('code_module', as_index = False) \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'success_try'}))
exams['success_percent'] =  round(exams.success_try / exams.total_try * 100, 2)
exams

,code_module,total_try,success_try,success_percent
0,CCC,1915,1683,87.89
1,DDD,3044,2709,88.99


***Всего по двум предметам были экзамены. По предмету CCC процент успешной сдачи ниже, чем по предмету DDD (87.89% против 88.99%)***

Исходя из задания не совсем понятно, нужно ли разбивать дополнительно экзамены по семестрам. 

In [11]:
exams_by_semestr = FullStudentAssessment.query('assessment_type == "Exam"') \
    .groupby(['code_module', 'code_presentation'], as_index = False) \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'total_try'}) \
    .merge(exam_ok.groupby(['code_module', 'code_presentation'], as_index = False) \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'success_try'}))
exams_by_semestr['success_percent'] =  round(exams_by_semestr.success_try / exams_by_semestr.total_try * 100, 2)
exams_by_semestr.sort_values('success_percent')

,code_module,code_presentation,total_try,success_try,success_percent
2,DDD,2013B,602,504,83.72
1,CCC,2014J,1168,1019,87.24
5,DDD,2014J,950,842,88.63
0,CCC,2014B,747,664,88.89
3,DDD,2013J,968,878,90.70
4,DDD,2014B,524,485,92.56


In [12]:
exams_by_semestr.success_percent.max()

92.56

***Самый низкий процент сдачи - по предмету DDD в семестре 2013B - 83.72 %***

***Самый высокий процент сдачи - предмету DDD в семестре 2014B - 92.56 %***



## Вопрос №3
По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (5 баллов) 

Отвечая на предыдущие вопросы, я уже отобрал ДФ только с успешными прохождениями экзаменов. Воспользуемся им:

In [13]:
exam_ok

,id_assessment,id_student,date_submitted,is_banked,score,code_module,code_presentation,assessment_type,date,weight
52924,24290,559706,234,0,78.0,CCC,2014B,Exam,NaN,100.0
52925,24290,559770,230,0,54.0,CCC,2014B,Exam,NaN,100.0
52926,24290,560114,230,0,64.0,CCC,2014B,Exam,NaN,100.0
52927,24290,560311,234,0,100.0,CCC,2014B,Exam,NaN,100.0
52928,24290,560494,230,0,92.0,CCC,2014B,Exam,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...
95978,25368,652617,243,0,56.0,DDD,2014J,Exam,NaN,100.0
95980,25368,652680,243,0,91.0,DDD,2014J,Exam,NaN,100.0
95981,25368,652732,249,0,47.0,DDD,2014J,Exam,NaN,100.0
95982,25368,652965,242,0,87.0,DDD,2014J,Exam,NaN,100.0


Определять средний срок сдачи экзаменов будем по колонке ***date_submitted*** — дата подачи заявки студентом, измеряемая как количество дней с начала семестра.

Сначала определим **средний срок сдачи с разбивкой по семестрам.**

In [14]:
exam_ok.groupby(['code_module', 'code_presentation'], as_index = False) \
    .agg({'date_submitted': 'mean'}) \
    .rename(columns={'date_submitted': 'mean_date'})

,code_module,code_presentation,mean_date
0,CCC,2014B,231.581325
1,CCC,2014J,244.404318
2,DDD,2013B,230.164683
3,DDD,2013J,239.509112
4,DDD,2014B,234.936082
5,DDD,2014J,242.804038


Определим **средний срок сдачи без разбивки по семестрам.** 

In [15]:
exam_ok.groupby(['code_module'], as_index = False) \
    .agg({'date_submitted': 'mean'}) \
    .rename(columns={'date_submitted': 'mean_date'})

,code_module,mean_date
0,CCC,239.345217
1,DDD,237.976006


## Вопрос №4
Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). (8 баллов)

In [16]:
studentRegistration.groupby(['code_module'], as_index = False) \
    .agg({'id_student': 'count'}) \
    .rename(columns={'id_student': 'reg_count'}) \
    .sort_values('reg_count', ascending = False) \
    .head(3)

,code_module,reg_count
1,BBB,7909
5,FFF,7762
3,DDD,6272


Самые популярные курсы (по кол-ву регистраций на них), без разбивки по семестрам - **BBB, FFF, DDD**

In [17]:
studentRegistration \
    .groupby(['code_module'], as_index = False) \
    .agg({'date_unregistration': 'count'})  \
    .rename(columns={'date_unregistration': 'unreg_count'}) \
    .sort_values('unreg_count', ascending = False) \
    .head(3)

,code_module,unreg_count
5,FFF,2380
1,BBB,2377
3,DDD,2235


Самый большой отток наблюдается с курсов **FFF, BBB, DDD**

## Вопрос №5
Построй адаптированные под задачу обучения RFM-кластеры студентов. Где **R — среднее время сдачи одного экзамена, F — завершаемость курсов, M — среднее количество баллов, получаемое за экзамен**. Подробно опиши, как ты создавал кластеры. 

**Строим кластер R для датафрейма**

In [18]:
FSA_R = FullStudentAssessment.query('assessment_type == "Exam"') \
    .groupby('id_student', as_index=False) \
    .agg({'date_submitted': 'mean'}) \
    .rename(columns={'date_submitted': 'exam_mean_time'})
FSA_R

,id_student,exam_mean_time
0,23698,243.0
1,24213,236.0
2,27116,243.0
3,28046,237.0
4,28787,243.0
...,...,...
4628,2694886,236.0
4629,2694933,230.0
4630,2695608,237.0
4631,2697181,230.0


Разбиваем на среднее время на 4 группы:  [0:24%], [25:49], [50-74], [75:100]

In [19]:
FSA_R['R'] = FSA_R.exam_mean_time\
    .apply(
    lambda x: 1 if x < FSA_R.exam_mean_time.quantile(0.25) 
    else (2 if x < FSA_R.exam_mean_time.quantile(0.5)
    else (3 if x < FSA_R.exam_mean_time.quantile(0.75) 
    else 4))
            )
FSA_R

,id_student,exam_mean_time,R
0,23698,243.0,4
1,24213,236.0,2
2,27116,243.0,4
3,28046,237.0,2
4,28787,243.0,4
...,...,...,...
4628,2694886,236.0,2
4629,2694933,230.0,1
4630,2695608,237.0,2
4631,2697181,230.0,1


**Строим кластер M для датафрейма**

Подобным образом разбиваем на 4 группы.

In [20]:
FSA_M = FullStudentAssessment.query('assessment_type == "Exam"') \
    .groupby('id_student', as_index=False) \
    .agg({'score': 'mean'})

FSA_M['M'] = FSA_M.score\
    .apply(
    lambda x: 1 if x < FSA_M.score.quantile(0.25)
    else (2 if x < FSA_M.score.quantile(0.5)
    else (3 if x < FSA_M.score.quantile(0.75)
    else 4))
    )
FSA_M

,id_student,score,M
0,23698,80.0,3
1,24213,58.0,2
2,27116,96.0,4
3,28046,40.0,1
4,28787,44.0,1
...,...,...,...
4628,2694886,69.0,3
4629,2694933,73.0,3
4630,2695608,73.0,3
4631,2697181,80.0,3


**Строим кластер F для датафрейма**

Считаем, сколько регистраций было у каждого студента на курсы:

In [21]:
registration_count = studentRegistration \
    .groupby('id_student', as_index=False) \
    .agg({'date_registration': 'count'}) \
    .rename(columns={'date_registration': 'registration_count'}) \
    .query('registration_count > 0')
registration_count

,id_student,registration_count
0,3733,1
1,6516,1
2,8462,2
3,11391,1
4,23629,1
...,...,...
28776,2698257,1
28777,2698535,2
28778,2698577,1
28779,2698588,1


Считаем, сколько экзаменов было у каждого студента:

In [22]:
exam_count = FullStudentAssessment \
    .query('assessment_type == "Exam"') \
    .groupby(['code_module', 'code_presentation', 'id_student'], as_index=False) \
    .agg({'id_assessment': 'count'}) \
    .rename(columns={'id_assessment': 'exam_count'})
exam_count

,code_module,code_presentation,id_student,exam_count
0,CCC,2014B,29764,1
1,CCC,2014B,29820,1
2,CCC,2014B,40604,1
3,CCC,2014B,42638,1
4,CCC,2014B,46605,1
...,...,...,...,...
4954,DDD,2014J,2677210,1
4955,DDD,2014J,2678380,1
4956,DDD,2014J,2684043,1
4957,DDD,2014J,2685863,1


Считаем соотношение экзаменов к регистрации, для дальнейшего сегментирования

In [23]:
FSA_F = registration_count \
    .merge(exam_count[['id_student', 'exam_count']], on=['id_student'], how='left') \
    .fillna(0) 
FSA_F['score'] = FSA_F.exam_count / FSA_F.registration_count
FSA_F

,id_student,registration_count,exam_count,score
0,3733,1,0.0,0.0
1,6516,1,0.0,0.0
2,8462,2,0.0,0.0
3,11391,1,0.0,0.0
4,23629,1,0.0,0.0
...,...,...,...,...
29076,2698257,1,0.0,0.0
29077,2698535,2,0.0,0.0
29078,2698577,1,0.0,0.0
29079,2698588,1,0.0,0.0


Пытаемся разбить на 4 группы, и получаем ерунду:

In [24]:
FSA_F['F'] = FSA_F.score\
    .apply(
    lambda x: 1 if x < FSA_F.score.quantile(0.25) 
    else (2 if x < FSA_F.score.quantile(0.5)
    else (3 if x < FSA_F.score.quantile(0.75) 
    else 4))
            )
FSA_F

,id_student,registration_count,exam_count,score,F
0,3733,1,0.0,0.0,4
1,6516,1,0.0,0.0,4
2,8462,2,0.0,0.0,4
3,11391,1,0.0,0.0,4
4,23629,1,0.0,0.0,4
...,...,...,...,...,...
29076,2698257,1,0.0,0.0,4
29077,2698535,2,0.0,0.0,4
29078,2698577,1,0.0,0.0,4
29079,2698588,1,0.0,0.0,4


25, 50, 75 процентили равны нулю. Значит, такое сегментирование нам не подойдет.

In [25]:
FSA_F.score.describe()

count    29081.000000
mean         0.142335
std          0.329203
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: score, dtype: float64

Так как минимальное значение 0, максимальное - 1, то разобьем просто на 4 группы, по равной четверти в каждой. (в группе 3, фактически, никогде нет в данном ДФ. Но ведь данные могут в любой момент пополниться, оставим деление на четыре группы, как в R и M)

In [26]:
FSA_F['F'] = FSA_F.score\
    .apply(
    lambda x: 1 if x <= 0.25 
    else (2 if x <= 0.5
    else (3 if x <= 0.75 
    else 4))
            )
FSA_F

,id_student,registration_count,exam_count,score,F
0,3733,1,0.0,0.0,1
1,6516,1,0.0,0.0,1
2,8462,2,0.0,0.0,1
3,11391,1,0.0,0.0,1
4,23629,1,0.0,0.0,1
...,...,...,...,...,...
29076,2698257,1,0.0,0.0,1
29077,2698535,2,0.0,0.0,1
29078,2698577,1,0.0,0.0,1
29079,2698588,1,0.0,0.0,1


***Ура, строим финальный датафрейм с разделением на кластеры:***

In [27]:
RFM = FSA_R[['id_student', 'R']] \
    .merge(FSA_F[['id_student', 'F']], on='id_student') \
    .merge(FSA_M[['id_student', 'M']], on='id_student')
RFM['RFM'] = RFM.R.astype(str) + RFM.F.astype(str) + RFM.M.astype(str)
RFM

,id_student,R,F,M,RFM
0,23698,4,4,3,443
1,24213,2,2,2,222
2,27116,4,4,4,444
3,28046,2,4,1,241
4,28787,4,4,1,441
...,...,...,...,...,...
4954,2694886,2,4,3,243
4955,2694933,1,4,3,143
4956,2695608,2,4,3,243
4957,2697181,1,4,3,143
